# Assingment 4 Testing and Stuff

$H(s)= - \sum_{i, j} J_{i, j} * s_i * s_j$

First step: load in data and interpret '+' and '-' as +1 and -1

In [43]:
import numpy as np
import os

cwd = os.getcwd()
ass_dir = cwd.rsplit('\\', maxsplit=1)[0]

#import
pm_data_str = np.loadtxt(os.path.join(ass_dir, r'data\in.txt'), dtype=str)

# separate string into characters
pm_data_sep = np.empty((pm_data_str.shape[0], len(pm_data_str[0])), dtype=str)
for i in range(len(pm_data_str)):
    pm_data_sep[i] = list(pm_data_str[i])

# locations of plus minus
p_loc = np.where(pm_data_sep == '+')
m_loc = np.where(pm_data_sep == '-')

# convert plus/minus to +1/-1
pm_data = np.empty(pm_data_sep.shape)
pm_data[p_loc] = 1.
pm_data[m_loc] = -1.
pm_data

array([[-1.,  1., -1.,  1.],
       [ 1.,  1.,  1.,  1.],
       [ 1., -1.,  1.,  1.],
       ...,
       [ 1.,  1.,  1.,  1.],
       [ 1., -1.,  1.,  1.],
       [-1.,  1., -1., -1.]])

The whole layout of this assignment is as follows:
1. Build Ising model that has a $\lambda_{i,j}$ which is the weights between nearest neighbours
2. Randomly (seeded) generate intial weights between -1 and 1 based on the size of Ising model (N weights for N atoms)
3. Use model to generate as many smaples as there are in the in.txt
4. Use model outputs and in.txt to compute gradient as follows:
$$
-\frac{\partial}{\partial \lambda_{i,j}} Loss (\lambda) = <s_i s_j>_D - <s_i s_j>_\lambda
$$
Where $s_i$ is the spin at particular location and you are averaging over the whole data set and generated set
5. Use the above gradient to update the weights of the model
6. Repeat steps 3-5 until model reaches the correct weights, $L$


Next we'll calcualte the average across the dataset

In [44]:
def avg_coupler(pm_data):
    data_avg_dict = {}
    data_avg_arr = np.empty(pm_data.shape[1])

    for j in range(pm_data.shape[1]):
        if j != pm_data.shape[1] - 1:
            data_avg_dict[(j, j+1)] = np.average(pm_data[:, j] * pm_data[:, j+1])
            data_avg_arr[j] = data_avg_dict[(j, j+1)]
        else:
            data_avg_dict[(j, 0)] = np.average(pm_data[:, j] * pm_data[:, 0])
            data_avg_arr[j] = data_avg_dict[(j, 0)]

    return data_avg_dict, data_avg_arr
avg_coupler(pm_data)

({(0, 1): -0.54, (1, 2): 0.466, (2, 3): 0.436, (3, 0): 0.454},
 array([-0.54 ,  0.466,  0.436,  0.454]))

The above roughly equals the {(0, 1): -1, (1, 2): 1, (2, 3): 1, (3, 0): 1} given to us as the true weights that produced the data set. This makes sense as those weights influence the monte carlo outputs. If it was truly random then we would expect ~0 for all.

Now we need to implement a 1D Ising model of size N

In [45]:
class Ising1D():
    '''
    1D Ising model class.
    '''
    def __init__(self, N, num_samples, seed=3141):

        # set random seed for generating weights
        np.random.seed(seed)
        self.weights = np.random.uniform(low=-1., high=1., size=N)

        self.N = N
        self.num_samples = num_samples

        # setup the lattices
        self.generate_lattices()
        

    def generate_lattices(self) -> np.ndarray:
        '''Generates num_samples amount of 1D lattices of shape N'''
        np.random.seed()
        self.lattices = (np.random.randint(0, 2, size=(self.num_samples, self.N)) * 2) - 1

    def equilibrium(self, flips_per_site=100):
        '''
        Lets each lattice go to equilibrium

        Params
        ---
        flips_per_site - average number of flips per site
        '''
        tot_flips = self.N * flips_per_site # total number of flips

        # random indices to attempt to flip
        np.random.seed()
        rand_j = np.random.randint(low=0, high=self.N, size=(self.num_samples, tot_flips))

        # outer loop goes through each lattice, letting each go to equilibrium
        for i in range(self.num_samples):

            # loop through the random indices trying to flip
            for j in rand_j[i, :]:
                # calculate current and new energy at indice j
                current, new = self.get_energy_difference(i, j)

                # comparing energies
                if current >= new:
                    # keep spin if improved or stayed same
                    self.lattices[i, j] = self.lattices[i, j] * -1

                if current < new:
                    # using metropolis algorithm we sometimes take this option
                    if np.random.random(1)[0] < np.exp(-(new - current)):
                        self.lattices[i, j] = self.lattices[i, j] * -1

    def get_energy_difference(self, i, j):
        '''Calculates and returns current and new energy at location j in lattice i'''

        # sum contributions from adjacent spins
        current = self.lattices[i, j] * self.lattices[i, j-1] * self.weights[j-1] * -1
        current += self.lattices[i, j] * self.lattices[i, (1+j-self.N)] * self.weights[1+j-self.N] * -1
        new = current * -1 # flipping of j is simply multiplying by negative 1

        return current, new

    def train_weights(self, train_data, num_epochs, learning_rate, flips_per_site=100):
        '''
        Trains weights based on data set
        '''
        train_coupler_avg = avg_coupler(train_data)

        for epoch in range(num_epochs):
            # generate lattices, go to equilibrium, and average couplers
            self.generate_lattices()
            self.equilibrium(flips_per_site=flips_per_site)
            model_coupler_avg = avg_coupler(self.lattices)

            # update weights 
            self.weights = self.weights + learning_rate * (train_coupler_avg[1] - model_coupler_avg[1])

            # we know weights can't be more than 1 or less than -1 so put a hard stop
            self.weights[np.where(self.weights > 1)] = 1.
            self.weights[np.where(self.weights < -1)] = -1. 

        print('Final weights: ', self.weights)

ising = Ising1D(pm_data.shape[1], pm_data.shape[0])
ising.train_weights(pm_data, 500, 0.05, flips_per_site=20)



Final weights:  [-1.  1.  1.  1.]


In [46]:
np.random.seed(3141)
weights = np.random.uniform(low=-1., high=1., size=4)
print(weights)
print(np.random.uniform(low=-1., high=1., size=4))
print(np.random.uniform(low=-1., high=1., size=4))


[-0.81941063  0.24702744 -0.20529057  0.66773735]
[ 0.68225318 -0.50259277 -0.57604001  0.58553827]
[ 0.81180796 -0.13352774 -0.85147106  0.80848687]


In [47]:
(np.random.randint(0, 2, size=(10, 4)) * 2) -1

array([[-1,  1, -1, -1],
       [ 1, -1, -1, -1],
       [-1, -1,  1,  1],
       [ 1, -1,  1, -1],
       [ 1,  1, -1,  1],
       [-1,  1, -1,  1],
       [-1,  1,  1, -1],
       [ 1,  1,  1,  1],
       [-1,  1, -1, -1],
       [ 1, -1,  1,  1]])